In [ ]:
!pip install opencv-python
!pip install mediapipe
!pip install numpy

In [ ]:
import cv2
import mediapipe as mp
import time
import serial

# 시리얼 통신 설정
arduino = serial.Serial('/dev/cu.usbmodem11101', 9600)  # 포트를 실제 연결된 아두이노 포트로 설정
time.sleep(2)  # 아두이노 연결 안정화 대기

# MediaPipe 손 인식 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

# 웹캠에서 이미지 캡처 및 해상도 설정
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

start_time = None
countdown = 3  # 카운트다운 시간 (초)
hand_shape_detected = None  # 감지된 손 모양

# 손가락 상태 확인 함수
def is_finger_up(finger_tip_idx, finger_pip_idx, landmarks):
    # DIP 관절을 사용하여 손가락이 접혔는지 확인
    return landmarks[finger_tip_idx].y < landmarks[finger_pip_idx].y

# 가위바위보 결과 판단 함수
def get_hand_shape(landmarks):
    index_up = is_finger_up(8, 6, landmarks)
    middle_up = is_finger_up(12, 10, landmarks)
    ring_up = is_finger_up(16, 14, landmarks)
    pinky_up = is_finger_up(20, 18, landmarks)

    if index_up and middle_up and not ring_up and not pinky_up:
        return "scissor"
    elif index_up and middle_up and ring_up and pinky_up:
        return "paper"
    elif not index_up and not middle_up and not ring_up and not pinky_up:
        return "rock"
    else:
        return "other"

def send_command_to_arduino(command):
    arduino.write(command.encode())

while cap.isOpened():
    success, image = cap.read()
    if not success:
        continue

    # 이미지 처리 및 손 감지
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Enter 키 감지 및 카운트다운 시작
    if cv2.waitKey(5) == 13:
        if start_time is None:
            start_time = time.time()
        else:
            hand_shape_detected = None
            start_time = None
            continue

    # 카운트다운 및 손 모양 감지
    if start_time:
        elapsed = time.time() - start_time
        remaining = countdown - int(elapsed)
        if remaining > 0:
            # 카운트다운 표시
            text = str(remaining)
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 2.0
            thickness = 5
            text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
            text_x = (image.shape[1] - text_size[0]) // 2
            text_y = (image.shape[0] + text_size[1]) // 2
            cv2.putText(image, text, (text_x, text_y), font, font_scale, (255, 255, 255), thickness, cv2.LINE_AA)
        elif remaining <= 0 and hand_shape_detected is None:
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    hand_shape_detected = get_hand_shape(hand_landmarks.landmark)
                    print(f"Detected Handshape: {hand_shape_detected}")
                    # 아두이노에 명령 전송
                    if hand_shape_detected == "scissor":
                        send_command_to_arduino('r')  # 가위 명령
                    elif hand_shape_detected == "rock":
                        send_command_to_arduino('p')  # 바위 명령
                    elif hand_shape_detected == "paper":
                        send_command_to_arduino('s')  # 보 명령

    # 결과 표시
    if hand_shape_detected:
        result_text = f"Result: {hand_shape_detected}"
        text_scale = 1.5
        text_thickness = 3
        text_size = cv2.getTextSize(result_text, cv2.FONT_HERSHEY_SIMPLEX, text_scale, text_thickness)[0]
        text_x = (image.shape[1] - text_size[0]) // 2
        text_y = (image.shape[0] + text_size[1]) // 2
        cv2.putText(image, result_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, text_scale, (0, 255, 0), text_thickness, cv2.LINE_AA)

    cv2.imshow('Hand Tracking', image)

    if cv2.waitKey(5) & 0xFF == 27:  # Esc key
        break

cap.release()
cv2.destroyAllWindows()
